In [9]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

# Load the dataset
file_path = '/Users/aliahmed/Downloads/archive/spotify_songs.csv'
data = pd.read_csv(file_path)

# Display the first few rows and the columns of the dataset
print(data.info())
print(data.head())

# Assuming the dataset has a 'track_name' column containing the titles of the songs
# Preprocess the titles
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(r'http\S+', '', text)  # Remove URLs
        text = re.sub(r'@\w+', '', text)  # Remove mentions
        text = re.sub(r'#\w+', '', text)  # Remove hashtags
        text = re.sub(r'\d+', '', text)  # Remove digits
        text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
        text = re.sub(r'\s+', ' ', text).strip()  # Replace multiple spaces with a single space
        text = ' '.join([word for word in text.split() if word not in stop_words])
    else:
        text = ""
    return text

data['cleaned_titles'] = data['track_name'].apply(preprocess_text)

# Check the cleaned titles
print(data[['track_name', 'cleaned_titles']].head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32833 entries, 0 to 32832
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   track_id                  32833 non-null  object 
 1   track_name                32828 non-null  object 
 2   track_artist              32828 non-null  object 
 3   track_popularity          32833 non-null  int64  
 4   track_album_id            32833 non-null  object 
 5   track_album_name          32828 non-null  object 
 6   track_album_release_date  32833 non-null  object 
 7   playlist_name             32833 non-null  object 
 8   playlist_id               32833 non-null  object 
 9   playlist_genre            32833 non-null  object 
 10  playlist_subgenre         32833 non-null  object 
 11  danceability              32833 non-null  float64
 12  energy                    32833 non-null  float64
 13  key                       32833 non-null  int64  
 14  loudne

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aliahmed/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
import gensim
import gensim.corpora as corpora
from pprint import pprint

# Tokenize the titles
def tokenize(text):
    return text.split()

data['tokens'] = data['cleaned_titles'].apply(tokenize)

# Create Dictionary
id2word = corpora.Dictionary(data['tokens'])

# Create Corpus
texts = data['tokens']

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])


[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)]]


In [13]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=10, 
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)

# Print the topics
pprint(lda_model.print_topics())


[(0,
  '0.050*"good" + 0.039*"feel" + 0.034*"better" + 0.034*"party" + 0.030*"sun" '
  '+ 0.029*"day" + 0.028*"im" + 0.026*"big" + 0.022*"knows" + 0.021*"without"'),
 (1,
  '0.169*"remix" + 0.093*"mix" + 0.061*"original" + 0.012*"night" + '
  '0.012*"one" + 0.012*"club" + 0.010*"go" + 0.008*"extended" + 0.007*"vs" + '
  '0.007*"take"'),
 (2,
  '0.048*"bad" + 0.034*"dawn" + 0.026*"war" + 0.021*"hours" + 0.020*"el" + '
  '0.017*"five" + 0.017*"cold" + 0.013*"bebe" + 0.013*"rexha" + '
  '0.012*"justin"'),
 (3,
  '0.098*"dont" + 0.071*"life" + 0.036*"dance" + 0.025*"sweet" + 0.023*"lose" '
  '+ 0.023*"control" + 0.023*"make" + 0.021*"wanna" + 0.020*"hands" + '
  '0.019*"nothing"'),
 (4,
  '0.227*"love" + 0.051*"get" + 0.028*"got" + 0.024*"beautiful" + '
  '0.018*"found" + 0.018*"tonight" + 0.018*"could" + 0.017*"higher" + '
  '0.017*"klaas" + 0.016*"kill"'),
 (5,
  '0.303*"feat" + 0.025*"back" + 0.022*"way" + 0.018*"u" + 0.012*"steve" + '
  '0.012*"know" + 0.012*"alive" + 0.011*"say" + 0.0

In [19]:
pip install pyLDAvis


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 3.4 MB/s eta 0:00:00
  Using cached numpy-1.26.4-cp311-cp311-macosx_11_0_arm64.whl.metadata (114 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 29.3 MB/s eta 0:00:00a 0:00:01
Using cached numpy-1.26.4-cp311-cp311-macosx_11_0_arm64.whl (14.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
Note: you may need to restart the kernel to use updated packages.


In [21]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.417135  0.149385       1        1  39.353748
5      0.076718 -0.252855       2        1  16.869348
6     -0.075377  0.051440       3        1   7.961452
4     -0.058977  0.047318       4        1   6.662567
9      0.050465 -0.270710       5        1   6.591368
0     -0.080733  0.053476       6        1   5.611376
3     -0.078564  0.054296       7        1   5.201530
8     -0.081919  0.055794       8        1   5.050630
7     -0.081793  0.056978       9        1   4.572503
2     -0.086954  0.054879      10        1   2.125477, topic_info=        Term         Freq        Total Category  logprob  loglift
21      feat  3990.000000  3990.000000  Default  30.0000  30.0000
6      remix  5198.000000  5198.000000  Default  29.0000  29.0000
33      edit  1724.000000  1724.000000  Default  28.0000  28.0000
35     radio  1471.000000  1471.000000  Default  27.0000  27.0000
34      love  1182.000000  1182.000000  Default  26.0000  26.0000
...      ...          ...          ...      ...      ...      ...
2050      se     3.807757     4.622821  Topic10  -6.0754   3.6572
969      qué     3.408664     4.223741  Topic10  -6.1861   3.6368
2776    rosa     3.311103     4.126476  Topic10  -6.2151   3.6310
4215  hoodie     3.256511     4.071571  Topic10  -6.2318   3.6278
2373   vassy     9.638108    12.622631  Topic10  -5.1467   3.5814

[339 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
2019      5  0.127373      adam
2019      6  0.866139      adam
575       2  0.989849  afrojack
233       4  0.981392      aint
480       9  0.956289       air
...     ...       ...       ...
91        3  0.996232     world
271       3  0.993396         x
1611      7  0.959397     years
529       1  0.998009     young
503       6  0.994997     youre

[318 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 6, 7, 5, 10, 1, 4, 9, 8, 3])

In [22]:
# Save the visualization as an HTML file
pyLDAvis.save_html(vis, 'lda_vis.html')
print("LDA visualization saved as lda_vis.html")


LDA visualization saved as lda_vis.html
